# Supplementary figure panels related to mutual information

To run this notebook, you need:
- Table of antigen EC$_{50}$s in a JSON file in `data/misc/`
- To have run `more_main_scripts/bootstrap_interpol_capacity.py`, `more_main_scripts/estimate_channel_capacity_cce.ipynb`, and `theoretical_antigen_classes_from_capacity_HighMI_3.ipynb` and saved their results in the folders `results/capacity/` (channel capacity results) and `results/fits/` (model parameter fits).  


Other MI-related panels are created by `main_plotting_scripts/peptide_channel_diagrams.py` and `theoretical_antigen_classes_from_capacity_HighMI_3.ipynb` directly. 



In [ ]:
import numpy as np
import scipy as sp
from scipy.interpolate import UnivariateSpline, PchipInterpolator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json

import os, sys
main_dir_path = os.path.abspath('../')
sys.path.insert(0, main_dir_path)

#from utils.distrib_interpolation import (eval_interpolated_means_covs, interpolate_params_vs_logec50, 
#                                         stats_per_levels, compute_cholesky_dataframe)
import utils.custom_pandas as custom_pd
from utils.extra_pairplots import dual_pairplot
from utils.plotting_mi import plot_params_vs_logec50
from utils.statistics import build_symmetric

In [ ]:
%matplotlib inline

In [ ]:
#plt.rcParams["figure.figsize"] = (2.25, 1.75)
plt.rcParams["axes.labelsize"] = 8.
plt.rcParams["legend.fontsize"] = 8.
plt.rcParams["axes.labelpad"] = 0.5
plt.rcParams["xtick.labelsize"] = 7.
plt.rcParams["ytick.labelsize"] = 7.
plt.rcParams["legend.title_fontsize"] = 8.
plt.rcParams["axes.titlesize"] = 8.
plt.rcParams["font.size"] = 8.
plt.rcParams["figure.dpi"] = 160

# Figure comparing fitted multivariate normal distributions to data

### Note: this figure is also produced directly in `compute_channel_capacity_HighMI_3.ipynb`


In [ ]:
foldr = os.path.join(main_dir_path, "results", "fits")
df_params = pd.read_hdf(os.path.join(foldr, "df_params_Sigmoid_freealpha_HighMI_3.hdf"))
foldr = os.path.join(main_dir_path, "results", "capacity")
suffix = "highmi3_Sigmoid_freealpha.hdf"
df_params_means = pd.read_hdf(os.path.join(foldr, "df_params_means_{}".format(suffix)))

df_params_means_estim_vari = pd.read_hdf(os.path.join(foldr, "df_params_means_estim_vari_{}".format(suffix)))
df_params_covs = pd.read_hdf(os.path.join(foldr, "df_params_covs_{}".format(suffix)))
df_params_covs_estim_vari = pd.read_hdf(os.path.join(foldr, "df_params_covs_estim_vari_{}".format(suffix)))
ser_npts = pd.read_hdf(os.path.join(foldr, "ser_npts_{}".format(suffix)))

# Also, the Cholesky decomposition
df_params_chol = pd.read_hdf(os.path.join(foldr, "df_params_chol_{}".format(suffix)))
df_params_chol_estim_vari = pd.read_hdf(os.path.join(foldr, "df_params_chol_estim_vari_{}".format(suffix)))

In [ ]:
df_ec50s = pd.read_json(os.path.join(main_dir_path, "data", "misc", "potencies_df_2021.json"))
df_ec50s.columns.name = "Reference"; df_ec50s.index.name = "Peptide"
ser_ec50s_avglog = np.log10(df_ec50s).mean(axis=1)
print(ser_ec50s_avglog)

In [ ]:
nsamples = 40
seed = 1357642
params_to_keep = ["a0", "tau0", "theta"]
levels_group = ["Peptide"]
rnd_gen = np.random.default_rng(seed=seed)
if len(levels_group) == 1:
    new_index = pd.MultiIndex.from_product([df_params_means.index] + [range(nsamples)], 
                                      names=[df_params_means.index.name, "Sample"])
else:
    new_index = pd.MultiIndex.from_product([*zip(*df_params_means.index)] + [range(nsamples)], 
                                      names=[df_params_means.index.names] + ["Sample"])
df_params_synth = pd.DataFrame(index=new_index, columns=params_to_keep, dtype=np.float64)
df_params_synth.columns.name = "Parameter"

# Sample from the fitted gaussians
for key in df_params_means.index:
    cov_mat = build_symmetric(df_params_covs.loc[key].values)
    mean_vec = df_params_means.loc[key].values
    df_params_synth.loc[key] = rnd_gen.multivariate_normal(mean_vec, cov_mat, nsamples)

In [ ]:
params_remove = list(set(df_params.index.names).difference(levels_group))
params_remove.remove("TCellNumber")
df_params_both = df_params.xs("30k", level="TCellNumber", axis=0).droplevel(params_remove).sort_index()

df_params_both = df_params_both.loc[:, params_to_keep[0]:params_to_keep[-1]]
print(df_params_both.groupby(levels_group).count().values)

idx = np.concatenate([np.arange(n) for n in df_params_both.groupby(levels_group).count().sort_index().values[:, 0]])
df_params_both["Sample"] = idx
df_params_both = df_params_both.set_index("Sample", append=True)
df_params_both = pd.concat([df_params_both, df_params_synth], axis=1, keys=["Data", "Synth"], names=["Source", "Parameter"])
df_params_both = df_params_both.stack("Source")
print(df_params_both)

In [ ]:
# Improve parameter labels
rename_dict = {"theta":r"$\theta$", "a0": r"$a_0$", "t0": r"$t_0$", "tau0":r"$\tau_0$"}
params_to_keep2 = [rename_dict[a] for a in params_to_keep]
peptides = ["N4", "Q4", "A2", "Y3", "T4", "V4", "G4", "E1"]
pep_color_order = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3", "A8", "Q7"]
pep_palette = {pep_color_order[i]:sns.color_palette()[i] for i in range(len(pep_color_order))}
palette_order = [pep_palette.get(a) for a in peptides]

In [ ]:
## NOTE: this figure is also produced directly in 
df_params_plot = custom_pd.xs_slice(df_params_both.rename(rename_dict, axis=1, level="Parameter"), 
                    name="Peptide", lvl_slice=peptides, axis=0).reset_index()

# Pairplot with reflection across the diagonal to compare synthetic vs data distributions
fig, axes, leg = dual_pairplot(data=df_params_plot, vari=params_to_keep2, 
    dual_lvl="Source", dual_labels=["Data", "Synthetic"], 
    dual_hues = [(0.5, 0.5, 0.5), plt.cm.viridis([206])[0]], palette=palette_order,
    hue="Peptide", hue_order=peptides, alpha=0.8, s=9, edgecolors=None)

# Clean up layout
fig.set_size_inches(4.75, 4.75)
fig.tight_layout(h_pad=0.5, w_pad=0.65)

# Uncomment to save supplementary figure
fig.savefig(os.path.join(main_dir_path, "figures", "capacity", 
    "pairplot_synthreal_dual_Sigmoid_freealpha_HighMI_3.pdf"), transparent=True, 
    bbox_extra_artists=(leg,), bbox_inches='tight', format="pdf")
plt.show()
plt.close()

# Figure showing interpolation of multivariate normal distribution parameters

In [ ]:
with open(os.path.join(main_dir_path, "results", "capacity", "ser_splines_means_capacity.pkl"), "rb") as hd:
    ser_splines_means = pd.Series(pickle.load(hd))
with open(os.path.join(main_dir_path, "results", "capacity", "ser_splines_chol_capacity.pkl"), "rb") as hd:
    ser_splines_chol = pd.Series(pickle.load(hd))
ser_splines_means.index.name = "Parameter"
ser_splines_chol.index.name = "Parameter"

In [ ]:
# Plot the interpolation of the means versus the data
df_params_plot = df_params_means.rename(rename_dict, axis=1, level="Mean element")
df_params_vari_plot = df_params_means_estim_vari.rename(rename_dict, axis=1, level="Var[Mean estimator]")
ser_splines_plot = ser_splines_means.rename(rename_dict, level="Parameter")

# Create a full subplots grid so all subplots will line up nicely in the end
fig, axes = plt.subplots(3, 3, sharey=False)
fig.set_size_inches(4.75, 4.5)

fig, axes_m = plot_params_vs_logec50(df_params_plot, df_params_vari_plot, ser_ec50s_avglog, 
                ser_interp=ser_splines_plot, cols_plot=None, x_name="Peptide", col_wrap=3, figax=[fig, axes[0]])
for ax in axes_m:  # 3 is col_wrap
    ax.set_xlabel(r"$\log_{10}{\mathrm{EC}_{50}}$ [-]")

# Uncomment to save figure
fig.tight_layout(w_pad=0.5)
fig.savefig(os.path.join(main_dir_path, "figures", "capacity", 
    "mean_vs_logec50_Sigmoid_freealpha_HighMI_3.pdf"), transparent=True, bbox_inches="tight")

# Second part of the plot: Cholesky elements
rename_dict2 = {"a0*a0":r"Chol[$a_0, a_0$]", "tau0*a0":r"Chol[$\tau_0, a_0$]", "theta*a0":r"Chol[$\theta, a_0$]", 
               "tau0*tau0":r"Chol[$\tau_0, \tau_0$]", "theta*tau0":r"Chol[$\theta, \tau_0$]", 
                "theta*theta":r"Chol[$\theta, \theta$]"}
df_params_plot = df_params_chol.rename(rename_dict2, axis=1, level="Cholesky element")
df_params_vari_plot = df_params_chol_estim_vari.rename(rename_dict2, axis=1, level="Var[Chol estimator]")
ser_splines_plot = ser_splines_chol.rename(rename_dict2, level="Parameter")

fig, axes_c = plot_params_vs_logec50(df_params_plot, df_params_vari_plot, ser_ec50s_avglog, 
                ser_interp=ser_splines_plot, cols_plot=None, x_name="Peptide", col_wrap=3, figax=[fig, axes[1:]])
for ax in axes_c:  # 3 is col_wrap
    ax.set_xlabel(r"$\log_{10}{\mathrm{EC}_{50}}$ [-]")

fig.tight_layout(w_pad=0.5)

# Uncomment to save figure
fig.savefig(os.path.join(main_dir_path, "figures", "capacity", 
            "means_cholesky_vs_logec50_Sigmoid_freealpha_HighMI_3.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Optimal distribution from CCE algorithm

#### Need to run `more_main_scripts/estimate_channel_capacity_cce.ipynb` and save the results before creating this panel. 

In [ ]:
## CHANGE THE DATE to when you executed that code and saved the result. 
with open(os.path.join(main_dir_path, "results", "capacity", 
            "cce_run_log_8ins_rtol4e-02_TCN30k_03-dec-2021.json"), "r") as hd:
    cce_run = json.load(hd)

In [ ]:
fig, ax = plt.subplots()
width = np.amin(np.abs(np.diff(cce_run["input_values"])))/2
ax.bar(cce_run["input_values"], cce_run["optim_input_distrib"], width=width, 
       color="xkcd:grey blue", edgecolor="k", linewidth=0.8)

# Labeling axes
ax.set_xticks(cce_run["input_values"])
ax.set_xticklabels(cce_run["input_peptides"])
ax.set_xlabel(r"$\log_{10}{(\mathrm{EC}_{50})}$", size=8)
ax.set_ylabel("P(EC$_{50}$)", size=8)
ax.tick_params(which="both", labelsize=6)
ax.annotate(r"$C = ({:.1f} \pm {:.1f})$ bits".format(cce_run["capacity_bits"], 0.04*cce_run["capacity_bits"]), 
            xy=(0.12, 0.85), xycoords="axes fraction", size=7)

# Invert EC50 axis to have N4 on the right
ax.invert_xaxis()

fig.set_size_inches(2.25, 2.)
fig.tight_layout()

plt.show()
plt.close()